## Please comment and upvote if you like this work!

## Take a look at the other notebooks in my profile:
## [CV notebooks](https://www.kaggle.com/killa92/code)

### Install libraries and Download a pre-trained Model

In [ ]:
from IPython import display
display.clear_output()

HOME = "/kaggle/working/"
!git clone https://github.com/SkalskiP/yolov9.git
%cd yolov9
!pip install -r requirements.txt -q
!pip install -q roboflow
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt

Cloning into 'yolov9'...
remote: Enumerating objects: 325, done.
remote: Total 325 (delta 0), reused 0 (delta 0), pack-reused 325
Receiving objects: 100% (325/325), 2.25 MiB | 29.11 MiB/s, done.
Resolving deltas: 100% (162/162), done.
/kaggle/working/yolov9


### Datasets and DataLoaders

In [ ]:
from glob import glob
import os

def make_ds(root, path_to_copy):
    os.makedirs(path_to_copy, exist_ok = True)
    os.makedirs(f"{path_to_copy}/dataset", exist_ok = True)
    
    files = glob(f"{root}/*")
    for idx, file in enumerate(files):
        if os.path.isdir(file): os.system(f"cp -r {file} {path_to_copy}")
        elif os.path.isfile(file): os.system(f"cp {file} {path_to_copy}")
            
root = "/kaggle/input/tomato-leaf-diseases-detection-computer-vision"
make_ds(root = root, path_to_copy = f"{HOME}/yolov9")

### Train YOLOv9

In [ ]:
%cd {HOME}/yolov9

!python train_dual.py \
--batch 4 --epochs 10 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
--data {HOME}/yolov9/data.yaml \
--weights {HOME}/weights/yolov9-c.pt \
--cfg models/detect/yolov9-c.yaml \
--hyp hyp.scratch-high.yaml 

### Visualization Learning Curves

In [ ]:
from PIL import Image
Image.open(f"{HOME}yolov9/runs/train/exp/confusion_matrix.png")

In [ ]:
Image.open(f"{HOME}yolov9/runs/train/exp/results.png")

### Validation Process

In [ ]:
%cd {HOME}/yolov9

!python val_dual.py \
--img 640 --batch 16 --conf 0.3 --iou 0.7 --device 0 \
--data {HOME}/yolov9/data.yaml \
--weights {HOME}/yolov9/runs/train/exp/weights/best.pt

### Inference

In [ ]:
!python detect.py \
--img 640 --conf 0.3 --device 0 \
--weights {HOME}/yolov9/runs/train/exp/weights/best.pt \
--source {HOME}/yolov9/test/images

### Inference Results Visualization

In [ ]:
from glob import glob
from PIL import Image
from matplotlib import pyplot as plt

for image_path in glob(f'{HOME}/yolov9/runs/detect/exp/*.jpg')[:5]:
      plt.imshow(Image.open(image_path)); plt.axis("off"); plt.show()
      print("\n")

In [ ]:
import streamlit as st
import joblib
import numpy as np

# Load the pre-trained model
model = joblib.load('model.pkl')

# Define the function to make predictions
def predict(values):
    values = np.array(values).reshape(1, -1)
    prediction = model.predict(values)
    return prediction

# Streamlit UI
st.title('Live Prediction Tool')

# Input fields
input_1 = st.number_input('Feature 1', value=0.0)
input_2 = st.number_input('Feature 2', value=0.0)
input_3 = st.number_input('Feature 3', value=0.0)

# Button to make prediction
if st.button('Predict'):
    inputs = [input_1, input_2, input_3]
    prediction = predict(inputs)
    st.write(f'The predicted value is: {prediction[0]}')
